In [ ]:
# Load Packages
import pandas as pd
import numpy as np
import plotly.express as px

import warnings
from typing import List
import os, sys, time

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import keras

rootpath = ".."
sys.path.insert(0, f"{os.getcwd()}/{rootpath}/base_models")
sys.path.insert(0, f"{os.getcwd()}/{rootpath}/source_models")
warnings.filterwarnings("ignore")

import model_prep


step_back = 6  # window size = 6*5 = 30 mins

step_back = 6  # window size = 6*5 = 30 mins
season_map = {
    "spring": [3, 4, 5],
    "summer": [6, 7, 8],
    "fall": [9, 10, 11],
    "winter": [12, 1, 2],
}

In [ ]:
from_building_name = "ESB"
from_tower_number = 1
to_building_name = "ESB"
to_tower_number = 2
features = ['FlowEvap', 'PerHumidity', 'TempAmbient', 'TempCondIn',
    'TempCondOut', 'TempEvapIn', 'TempEvapOut', 'TempWetBulb',
    'PerFreqConP', 'Tonnage', 'PerFreqFan']
target = 'EnergyConsumption'
to_season = "summer"
from_season = "summer"
finetuning_percentage = 0.8
source_epochs=100
finetune_epochs = 100
display_results = True
use_delta = True
shuffle_seed = 42
train_percentage = 0.8

In [ ]:
"""
1. Load data and do basic preprocessing
"""
# load data
df = pd.read_csv(
    f"{rootpath}/data/{from_building_name.lower()}/{from_building_name.lower()}{from_tower_number}_preprocessed.csv",
    index_col="time",
)
df.index = pd.to_datetime(df.index)

# only take data for one season
df = model_prep.choose_season(df, season=from_season)

# remove cases in which tower was OFF, and cases where OFF data would be included in past timesteps of ON data
on_condition = df[target] > 0
df = df.drop(df[~on_condition].index, axis=0)

# select features and targets and create final dataframe that includes only relevant features and targets
df = df[features+["DayOfWeek"]].join(df[target], on=df.index)

# if difference from first value should be used as for predictions then return the first value
first_val = df.iloc[0, df.columns.get_loc(target)]
if use_delta:
    df[target] = (
        df[target] - first_val
    )

In [ ]:
"""
2a. Trend removal on target
"""
def calculate_tsi(time_series, m):
    """
    Calculate ts_i for each data point i in the consumption time series.

    Parameters:
    - time_series: Pandas Series representing the consumption time series.
    - m: Integer representing the number of data points to consider before i.

    Returns:
    - Pandas Series containing ts_i values.
    """

    # Initialize an empty list to store the calculated ts_i values
    ts_i_values = []

    # Iterate through the time series
    for i in range(len(time_series)):
        if i < m:
            # For the first m data points, use the mean of available data
            ts_i = time_series.iloc[:i + 1].mean()
        else:
            # For subsequent data points, calculate ts_i using the formula
            ts_i = 1 / m * time_series.iloc[i - m + 1:i + 1].sum()

        # Append the calculated ts_i value to the list
        ts_i_values.append(ts_i)

    # Create a Pandas Series from the list of ts_i values
    ts_i_series = pd.Series(ts_i_values, index=time_series.index, name='ts_i')

    return ts_i_series


In [ ]:
# def invert_tsi(results, m, old_df_len):
#     """
#     Calculate ts_i for each data point i in the consumption time series.

#     Parameters:
#     - time_series: Pandas Series representing the consumption time series.
#     - m: Integer representing the number of data points to consider before i.

#     Returns:
#     - Pandas Series containing ts_i values.
#     """

#     # Initialize an empty list to store the calculated ts_i values
#     ts_i_values = []

#     # Iterate through the time series
#     for i in range(len(results)):
#         if i < m:
#             # For the first m data points, use the mean of available data
#             ts_i = results.iloc[:i + 1].mean()
#         else:
#             # For subsequent data points, calculate ts_i using the formula
#             ts_i = 1 / m * time_series.iloc[i - m + 1:i + 1].sum()

#         # Append the calculated ts_i value to the list
#         ts_i_values.append(ts_i)

#     # Create a Pandas Series from the list of ts_i values
#     ts_i_series = pd.Series(ts_i_values, index=time_series.index, name='ts_i')

#     return ts_i_series
    

In [ ]:
"""
2b. Seasonality removal
"""

def calculate_seasonal_index(time_series, seasonality_column, m):
    """
    Calculate the seasonal index for each seasonality value in the time series.

    Parameters:
    - time_series: Pandas DataFrame containing the time series data with a column for the seasonality values.
    - seasonality_column: String representing the column name containing the seasonality values (e.g., days of the week).
    - m: Integer representing the number of data points for each seasonality value.

    Returns:
    - Pandas DataFrame containing the seasonal index for each seasonality value.
    """

    # Group the data by the seasonality column
    grouped_data = time_series.groupby(seasonality_column)

    # Calculate the average of all target variable data points
    y_bar = time_series.mean()[target]

    # Initialize an empty dictionary to store the seasonal index values
    seasonal_index_dict = {}

    # Iterate through each group (seasonality value)
    for group, group_data in grouped_data:
        # Calculate the sum of the first m data points
        sum_y_p_j = group_data.iloc[:m][target].sum()

        # Calculate the seasonal index using the provided formula
        seasonal_index = 1 / y_bar * (1 / m) * sum_y_p_j

        # Store the seasonal index value in the dictionary
        seasonal_index_dict[group] = seasonal_index

    # Convert the dictionary to a Pandas DataFrame
    seasonal_index_df = pd.DataFrame(list(seasonal_index_dict.items()), columns=[seasonality_column, 'sp'])

    return seasonal_index_df

def operate_with_sp(col, sp_df, operation):
    index_col = col.index
    combined_df = pd.merge(col, sp_df, left_on=col.index.dayofweek, right_on='DayOfWeek', how='left').set_index(index_col)
    if operation == 'multiply':
        combined_df[col.name] = combined_df[col.name] * combined_df['sp']
    elif operation == 'divide':
        combined_df[col.name] = combined_df[col.name] / combined_df['sp']
    else:
        raise ValueError('Invalid operation')
    return combined_df[col.name]

In [ ]:
# apply trend removal: UNDONE
# m = 7*int(24*60/5) # FIXME: choose this carefully
# df['ts_i'] = calculate_tsi(df[target], m)
# df['tf_i'] = df['ts_i'] # chose len(df), bcs paper says to pick last index but my indices are time objects

# df[target] = df[target] / df['tf_i']

# apply seasonality removal
sdf = calculate_seasonal_index(df, 'DayOfWeek', 7)
df[target] = operate_with_sp(df[target], sdf, 'divide')

In [ ]:
df[target].plot()

In [ ]:
"""
3. Split data into training and testing sets
"""

df = df.dropna() # drop first NaN value due to zero division
X = df[features]  # only have features
y = df[target]  # only have target column

# split into input and outputs
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=(1 - train_percentage), shuffle=False, random_state=shuffle_seed
)

# scale feature data
scaler = MinMaxScaler().fit(X_train)
X_train[X_train.columns] = scaler.transform(X_train)
X_test[X_test.columns] = scaler.transform(X_test)
vec_X_train = X_train.values
vec_X_test = X_test.values


vec_y_train = y_train.values
vec_y_test = y_test.values
# scaler_y = MinMaxScaler().fit(y_train.values.reshape(-1, 1))
# vec_y_train = scaler_y.transform(y_train.values.reshape(-1, 1))
# vec_y_test = scaler_y.transform(y_test.values.reshape(-1, 1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense


# Build the MLP model
model = Sequential()
model.add(Dense(20, input_shape=(len(features),), kernel_initializer='normal', activation='relu'))
model.add(Dense(units=1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0) # FIXME: optimize hyperparams

In [ ]:
# Make predictions on the test set
y_pred = model.predict(vec_X_test)

# Inverse transform to get predictions in the original scale
# y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
# vec_y_test = scaler_y.inverse_transform(vec_y_test.reshape(-1, 1))

In [ ]:
# Evaluate the model

results_df = pd.DataFrame(
    {
        "actual": vec_y_test.reshape((vec_y_test.shape[0])),
        "predicted": y_pred.reshape((y_pred.shape[0])),
    },
    index=y_test.index,
)

# invert seasonality removal
results_df['actual'] = operate_with_sp(results_df['actual'], sdf, 'multiply')
results_df['predicted'] = operate_with_sp(results_df['predicted'], sdf, 'multiply')

# invert trend removal UNDONE
# merged_df = pd.merge(results_df, df['tf_i'], left_on=results_df.index, right_on=df['tf_i'].index, how='left').set_index(results_df.index)
# merged_df['actual'] = merged_df['actual'] * merged_df['tf_i']
# merged_df['predicted'] = merged_df['predicted'] * merged_df['tf_i']
# merged_df.drop(columns=[col for col in merged_df.columns if col not in ["actual", "predicted"]], inplace=True)
# results_df = merged_df

mae = mean_absolute_error(results_df['actual'], results_df['predicted'])
print(f"Mean Absolute Error: {mae}")

In [ ]:
# Plot the results
display_df = pd.DataFrame(
    index=pd.date_range(
        start=results_df.index.min(), end=results_df.index.max(), freq="5min"
    )
).merge(results_df, how="left", left_index=True, right_index=True)


fig = px.line(display_df, x=display_df.index, y=["actual", "predicted"])
fig

In [ ]:
import matplotlib.pyplot as plt
# Plot with reduced opacity
ax = results_df.plot(alpha=0.5)

# Customize the plot if needed (e.g., add labels, title, etc.)
ax.set_xlabel('X-axis Label')
ax.set_ylabel('Y-axis Label')
ax.set_title('Title')

# Show the plot
plt.show()

# Transfer Logic

In [ ]:
"""
1. Load data and do basic preprocessing
"""
# load data
df = pd.read_csv(
    f"{rootpath}/data/{to_building_name.lower()}/{to_building_name.lower()}{to_tower_number}_preprocessed.csv",
    index_col="time",
)
df.index = pd.to_datetime(df.index)

# only take data for one season
df = model_prep.choose_season(df, season=from_season)

# remove cases in which tower was OFF, and cases where OFF data would be included in past timesteps of ON data
on_condition = df[target] > 0
df = df.drop(df[~on_condition].index, axis=0)

# select features and targets and create final dataframe that includes only relevant features and targets
df = df[features + ["DayOfWeek"]].join(df[target], on=df.index)

# if difference from first value should be used as for predictions then return the first value
first_val = df.iloc[0, df.columns.get_loc(target)]
if use_delta:
    df[target] = (
        df[target] - first_val
    )

In [ ]:
"""
3. Split data into training and testing sets
"""

# Calculate the index to split the data (% training, % testing)
num_rows = len(df)
split_index = int(finetuning_percentage * num_rows)

# Split the data
train_set = df.iloc[:split_index]
test_set = df.iloc[split_index:]

In [ ]:
# apply trend removal UNDONE
# m = 7*int(24*60/5) # FIXME: choose this carefully
# train_set['ts_i'] = calculate_tsi(train_set[target], m)
# tf_len_multiplier = 1 # len(train_set)
# train_set['tf_i'] = train_set['ts_i'] # EDITT len(train_set) # chose len(df), bcs paper says to pick last index but my indices are time objects
# train_set[target] = train_set[target] / train_set['tf_i']

# apply seasonality removal
sdf = calculate_seasonal_index(train_set, 'DayOfWeek', 7)
train_set[target] = operate_with_sp(col=train_set[target], sp_df=sdf, operation="divide")

In [ ]:
# drop nan values due to trend/seasonality removal
train_set = train_set.dropna()
test_set = test_set.dropna()

# Split further
X_train, X_test = train_set[features], test_set[features]
y_train, y_test = train_set[target], test_set[target]

In [ ]:
# scale feature data - use source domain scaler
X_train[X_train.columns] = scaler.transform(X_train)
X_test[X_test.columns] = scaler.transform(X_test)
vec_X_train = X_train.values
vec_X_test = X_test.values


vec_y_train = y_train.values
vec_y_test = y_test.values

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)

In [ ]:
# Evaluate the model
y_pred = model.predict(vec_X_test)

# Evaluate the model

results_df = pd.DataFrame(
    {
        "actual": vec_y_test.reshape((vec_y_test.shape[0])),
        "predicted": y_pred.reshape((y_pred.shape[0])),
    },
    index=y_test.index,
)

In [ ]:
# invert seasonality removal
results_df['actual'] = operate_with_sp(results_df['actual'], sdf, 'multiply')
results_df['predicted'] = operate_with_sp(results_df['predicted'], sdf, 'multiply')

In [ ]:
# # invert trend removal UNDONE
# def recover_original_values(dtframe, target_column, m, start_index, tf_multiplier):
#     # Create a new column to store the recovered original values
#     recovered_column = f"recovered_{target_column}"
#     dtframe[recovered_column] = dtframe[target_column]

#     for i, (index, _) in enumerate(dtframe.loc[start_index:].iterrows()):
        
#         # Calculate the sum of the past m-1 values
#         past_values_sum = dtframe[target_column].iloc[i - m + 1:i].sum()
#         # Calculate the recovered original value by subtracting the sum from the average
#         recovered_value = dtframe[target_column].iloc[i] * m - past_values_sum
#         # Update the recovered column with the original value
#         dtframe.at[index, recovered_column] = recovered_value
    
#     dtframe[f"recovered_{target_column}"] = dtframe[f"recovered_{target_column}"] / tf_multiplier

#     return dtframe


# pred_set = test_set.copy()
# pred_set[target] = results_df["predicted"]
# start_of_predictions = pred_set.index[0]
# combined_train_pred = train_set.append(pred_set, ignore_index=False)
# rec_train_pred = recover_original_values(combined_train_pred, target, 7, start_of_predictions, tf_len_multiplier)
# results_df['predicted'] = rec_train_pred[start_of_predictions:][f"recovered_{target}"]

In [ ]:
mae = mean_absolute_error(results_df['actual'], results_df['predicted'])
print(f"Mean Absolute Error: {mae}")

# Plot the results
display_df = pd.DataFrame(
    index=pd.date_range(
        start=results_df.index.min(), end=results_df.index.max(), freq="5min"
    )
).merge(results_df, how="left", left_index=True, right_index=True)


fig = px.line(display_df, x=display_df.index, y=["actual", "predicted"])
fig